![](https://i.ytimg.com/vi/tyyA0VtQXjQ/maxresdefault.jpg)

 # Projet web Mining : 
 # FEUDJIO NGOUANET VALDES VOTEL   &   SOBJIO LUDANE LAGNOL    


### Introduction au projet

Dans le cadre des travaux pratiques du cours de Traitement Automatique des Langues (TAL), nous avons pour objectif de concevoir une application dédiée à l'extraction d'informations à partir de données issues de flux RSS. Ce projet vise à mettre en œuvre des techniques fondamentales du TAL pour collecter, analyser et extraire des connaissances pertinentes à partir d'un corpus de textes en français.

L'application s'articule en deux grandes étapes :
1. **Scrapping et collecte de données** : Cette phase consiste à capturer des articles via des flux RSS, en les intégrant dans une base de données structurée. Elle repose sur des outils tels que `feedparser` et `newspaper` pour récupérer les métadonnées et le contenu textuel des articles.
2. **Extraction et analyse d'informations** : À partir du corpus collecté, nous explorons des techniques de reconnaissance des entités nommées (Personnes, Organisations, Lieux, etc.) et d'identification des relations entre ces entités. Cette tâche, connue sous le nom d'extraction d'information, permet de révéler les interactions pertinentes au sein du corpus.

Les enjeux de ce projet sont multiples :
- Automatiser la collecte de données textuelles à partir de flux en ligne.
- Appliquer des modèles linguistiques pour identifier des informations structurées dans des textes non structurés.
- Comprendre les relations sémantiques au sein des données pour dégager des connaissances exploitables.

Les résultats attendus incluent :
- Une base de données enrichie par des articles analysés et leurs métadonnées.
- Une analyse qualitative des entités nommées et des relations détectées.
- Une réflexion critique sur les méthodes utilisées, leurs limites et des pistes d’amélioration.

Ce projet illustre des applications concrètes du TAL, notamment dans les domaines de la veille géopolitique, économique ou encore du fact-checking journalistique. À travers cette démarche, nous développons une expertise technique et conceptuelle sur des outils et méthodes clés dans l'extraction et l'analyse de l'information.

### PARTIE 1 : WEB SCRAPING DE LA BASE DE DONNEES

#### installation des packages de base

In [10]:
#!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/francetvinfo.json
#!wget https://people.irisa.fr/Guillaume.Gravier/teaching/ENSAI/data/tvinfo-sources.json
#!pip install feedparser
#!pip install newspaper3k
#!python -m spacy download fr_core_news_md

#### Chargement des données deja existantes et des données sur les flus RSS

In [11]:
import json
import ssl
import feedparser as fp

# (a) Se prémunir contre le blocage de commande
ssl._create_default_https_context = ssl._create_unverified_context

# (b) Lire la base de données existante (francetvinfo.json)
with open("francetvinfo.json", "r", encoding="utf-8") as f:
    existing_data = json.load(f)


# (c) Lire les flux RSS listés dans tvinfo-sources.json
with open("tvinfo-sources.json", "r", encoding="utf-8") as f:
    rss_sources = json.load(f)


#### Selection des données qui ne sont pas encore dans la base de données existante

# QUESTION 2

Pour identifier les zones contenant le titre et le texte d'un article dans le code HTML d'une page web, il faut d'abord inspecter la structure de la page.

1. **Le titre de l'article** : Le titre se trouve souvent dans un élément `<h1>`, `<h2>`, ou un autre type de balise avec une classe spécifique (par exemple, `<h1 class="article-title">`). En inspectant le code HTML, on peut repérer cette balise pour obtenir le titre de l'article.

2. **Le texte de l'article** : Le texte principal de l'article est généralement contenu dans un bloc `<div>`, `<article>`, ou parfois dans des balises `<p>` à l'intérieur d'une section avec une classe désignant le corps de l'article, comme `<div class="content">` ou `<div class="article-body">`. Vous devez repérer ces éléments pour récupérer le contenu principal.

### Que faut-il faire pour récupérer le texte de l'article ?
Pour récupérer le texte de l'article, vous devrez analyser le code HTML et localiser les éléments correspondants (généralement des balises `<div>`, `<article>`, `<section>`, ou des balises spécifiques contenant des classes comme "content" ou "body"). Ensuite, vous pouvez extraire le texte de ces éléments en utilisant des outils comme BeautifulSoup en Python ou d'autres méthodes de parsing HTML.

### L'utilité de la bibliothèque Newspaper
La bibliothèque *newspaper3k* est utile car elle automatise cette tâche de récupération du texte de l'article en analysant et extrayant directement le contenu principal de la page HTML. Elle est particulièrement efficace pour contourner les variations dans la structure HTML d'une page et permet de récupérer rapidement le titre, l'auteur, la date de publication et le texte sans avoir à inspecter manuellement chaque balise HTML. 

*Newspaper3k* fournit également des fonctionnalités supplémentaires comme la gestion des images, le nettoyage du contenu (en retirant les publicités ou les éléments non pertinents), et l'analyse sémantique du texte. Cela facilite grandement l'extraction d'informations de manière fiable et automatisée, même pour des pages au design complexe ou changeant.

# QUESTION 3

In [12]:
import feedparser as fp
from newspaper import Article
import requests
from urllib.parse import urlparse

# Extraire les liens existants pour éviter les doublons
existing_links = existing_data.keys()

# Scanner les flux RSS et repérer les nouveaux articles
new_articles = {}
invalid_urls_count = 0  # Compteur pour les URLs invalides

# Fonction pour vérifier si l'URL du flux RSS est valide

"""
Ce bloc de code parcourt les articles d'un flux RSS et vérifie si chaque article est nouveau en comparant son lien avec une liste de liens existants.
Pour chaque nouvel article, il utilise la bibliothèque `newspaper` pour télécharger et analyser les informations détaillées de l'article.
Les détails extraits, tels que le titre, la date de publication, l'auteur, la catégorie, le contenu et le lien de l'image, sont ensuite ajoutés à un dictionnaire `new_articles`.
En cas d'erreur de connexion ou d'autre exception lors du traitement de l'article, un message d'erreur est affiché.
"""


def check_url_validity(url):
    try:
        if not isinstance(url, str):  # Vérifier que l'URL est bien une chaîne de caractères
            print(f"URL invalide (pas une chaîne de caractères) : {url}")
            return False

        # Vérifier que l'URL ne contient pas de fragments ou autres éléments invalides
        parsed_url = urlparse(url)
        if parsed_url.fragment:
            print(f"URL invalide (contenant un fragment) : {url}")
            return False  # Si l'URL contient un fragment, on la considère invalide

        response = requests.get(url, timeout=5)
        if response.status_code != 200:
            print(f"Erreur de connexion pour l'URL {url} : Code de statut {response.status_code}")
            return False
        
        return True
    except requests.exceptions.RequestException as e:
        print(f"Erreur de connexion pour l'URL {url}: {e}")
        return False

for category, url in rss_sources.items():
    print(f"Lecture du flux RSS '{category}': {url}")
    
    # Vérification de la validité de l'URL
    if not check_url_validity(url):
        invalid_urls_count += 1  # Incrémenter le compteur pour les URLs invalides
        continue  # Passer au flux suivant

    # Analyser le flux RSS
    data = fp.parse(url)

    # Parcourir les articles du flux RSS
    for item in data.entries:
        if item.link not in existing_links:  # Vérifier si l'article est nouveau
            try:
                # Utiliser newspaper pour extraire les informations détaillées
                article = Article(item.link)
                article.download()
                article.parse()

                # Ajouter les détails de l'article dans new_articles
                new_articles[item.link] = {
                    "title": article.title or item.title,
                    "date": article.publish_date if article.publish_date else (item.published if hasattr(item, "published") else "N/A"),
                    "author": article.authors or item.get("author", "Inconnu"),
                    "category": category,
                    "content": article.text or item.get("description", "Pas de contenu"),
                    "image_link": article.top_image or "Pas d'image"
                }

            except requests.exceptions.RequestException as e:
                print(f"Erreur de connexion pour l'article {item.link}: {e}")
            except Exception as e:
                print(f"Erreur lors du traitement de l'article {item.link} : {e}")

# Afficher les résultats
print(f"{len(new_articles)} nouveaux articles")
print(f"{invalid_urls_count} flux RSS invalides ou inaccessibles.")


Lecture du flux RSS 'france': https://www.francetvinfo.fr/france.rss
Lecture du flux RSS 'europe': https://www.francetvinfo.fr/monde/europe.rss
Lecture du flux RSS 'economie': https://www.francetvinfo.fr/economie/entreprises.rss
59 nouveaux articles
0 flux RSS invalides ou inaccessibles.


### Résumé du comportement

- Les flux RSS sont parcourus et les articles sont récupérés.
- Les doublons sont évités en vérifiant si le lien de l'article existe déjà dans `existing_links`.
- Les informations détaillées de chaque article sont extraites via la bibliothèque `newspaper3k`, et elles sont ajoutées à `new_articles`.
- Les erreurs sont gérées avec un bloc `try-except`, ce qui permet de continuer l'exécution même en cas de problème avec un article spécifique.

In [13]:
# Afficher les 10 premiers liens des nouveaux articles
print("Nombres d'articles",len(new_articles))
for key in list(new_articles.keys())[:10]:
    print(key)

Nombres d'articles 59
https://www.francetvinfo.fr/france/hauts-de-france/pas-de-calais/pas-de-calais-au-moins-trois-migrants-morts-dans-une-tentative-de-traversee-de-la-manche_6983336.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/faits-divers/police/marseille-des-policiers-vises-par-des-tirs-cette-nuit-lors-d-une-course-poursuite_6983255.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/france/hauts-de-france/pas-de-calais-au-moins-deux-morts-dans-une-tentative-de-traversee-de-la-manche_6983165.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/monde/environnement/video-trier-ses-dechets-est-il-vraiment-utile_6979421.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/monde/europe/migrants/au-moins-trois-migrants-sont-morts-dans-le-pas-de-calais-lors-d-une-tentative-de-traversee-de-la-manche_6983171.html#xtor=RSS-3-[general]
https://www.francetvinfo.fr/replay-radio/le-journal-des-outre-mers/mayotte-toujours-dans-la-detresse-15-jours-apres-le-passage-du-cyclone-chido_69

In [14]:
import json
from datetime import datetime

# Fonction pour convertir les objets datetime en chaîne de caractères
def convert_datetime(obj):
    if isinstance(obj, datetime):
        return obj.strftime('%Y-%m-%d %H:%M:%S')  # Vous pouvez adapter le format à vos besoins
    raise TypeError("Type non sérialisable")

# Charger les données existantes dans tvinfo-sources.json
try:
    with open("francetvinfo.json", "r", encoding="utf-8") as file:
        existing_data = json.load(file)
except FileNotFoundError:
    print("Le fichier francetvinfo.json n'existe pas encore, il sera créé.")
    existing_data = {}  # Initialiser un dictionnaire vide si le fichier n'existe pas

# Mettre à jour les données existantes avec les nouveaux articles
existing_data.update(new_articles)

# Sauvegarder les données mises à jour dans tvinfo-sources.json
with open("francetvinfo.json", "w", encoding="utf-8") as file:
    # Utiliser `default` pour convertir les objets `datetime`
    json.dump(existing_data, file, indent=4, ensure_ascii=False, default=convert_datetime)

# Afficher le nombre d'articles ajoutés
added_articles_count = len(new_articles)
print(f"{added_articles_count} nouveaux articles ont été ajoutés au fichier francetvinfo.json.")
print("Le fichier francetvinfo.json a été mis à jour.")


59 nouveaux articles ont été ajoutés au fichier francetvinfo.json.
Le fichier francetvinfo.json a été mis à jour.


Le code ci-dessus a pour objectif de mettre à jour une base de données d'articles en y ajoutant de nouveaux articles récupérés depuis plusieurs flux RSS. Il commence par réinitialiser la base de données existante, puis il ajoute les nouveaux articles détectés, en évitant les doublons grâce à une vérification des liens. Chaque article est analysé pour extraire ses détails (titre, date, auteur, contenu, image, etc.) à l'aide de la bibliothèque *newspaper3k*. Enfin, la base de données mise à jour contient l'ensemble des articles existants et les nouveaux, et la taille totale de la base peut être affichée.

# QUESTION 4
Pour gérer les images et garantir qu'elles restent accessibles même si elles sont supprimées du site web, voici comment nous pourrions procéder :

1. **Téléchargement de l'image** :  
   - Lors de l'extraction des articles via la bibliothèque *newspaper3k*, nous récupérons souvent l'URL de l'image principale avec l'attribut `article.top_image`. Nous pouvons utiliser cette URL pour télécharger l'image.
   - Nous utiliserons une bibliothèque telle que `requests` pour télécharger l'image depuis l'URL fournie.

2. **Stockage local des images** :  
   - Une fois l'image téléchargée, nous la sauvegardons dans un répertoire spécifique de notre système de fichiers, par exemple dans un dossier appelé `images` ou `media`.
   - Le nom du fichier peut être dérivé du titre de l'article ou d'un identifiant unique (comme un hash de l'URL de l'image) pour éviter les conflits de noms de fichiers.

3. **Référence dans la base de données** :  
   - Plutôt que de conserver uniquement l'URL de l'image dans la base de données, nous enregistrons le chemin local du fichier sur le disque (par exemple, `./images/article123.jpg`).
   - Cela nous permettra de nous y référer ultérieurement, même si l'image originale a été supprimée du site.

4. **Vérification régulière** :  
   - Pour gérer l'espace disque et s'assurer que les images locales sont toujours pertinentes, nous pourrions mettre en place une tâche planifiée pour vérifier périodiquement si les articles sont toujours valides et si les images sont toujours présentes sur le site. Si elles sont manquantes, nous pourrions décider de supprimer l'image locale ou de la mettre à jour.

5. **Sécurisation et gestion des tailles** :  
   - Pour éviter de stocker des images trop volumineuses, nous pourrions éventuellement compresser les images avant de les stocker.
   - Il pourrait également être utile de renommer les fichiers avec un format standard et de les convertir en un format d'image plus léger, comme le format `.webp`, si nécessaire.

En suivant cette approche, nous garantirions que les images restent accessibles localement, même si elles sont supprimées ou déplacées sur le site web d'origine.
# QUESTION 5

Pour collecter des informations complémentaires sur les articles que nous avons extraits des flux RSS, nous pourrions explorer d'autres sources d'information en ligne telles que :

### 1. **Twitter (ou autres réseaux sociaux)**
   - **Ce que nous pourrions collecter** :
     - **Réactions et discussions** autour de l'article, incluant des commentaires et des retweets, qui peuvent fournir un aperçu de la réception publique de l'article ou de l'événement couvert.
     - **Hashtags et mentions** associés à l'article ou au sujet, afin de suivre l'actualité et voir comment elle évolue sur les réseaux sociaux.
     - **Auteur de tweets** ou **influenceurs** qui partagent ou commentent l'article, permettant d'étendre la recherche à des points de vue ou des perspectives différentes.
   
   - **Librairies utiles** :
     - `Tweepy` ou `TwitterAPI` : Ces bibliothèques Python nous permettent de collecter des tweets en temps réel ou via l'API de recherche, en ciblant des mots-clés, des hashtags, ou des mentions liées à l'article.
   
### 2. **Google News API**
   - **Ce que nous pourrions collecter** :
     - **Articles similaires** à celui que nous avons récupéré, en utilisant des mots-clés pour rechercher des sources complémentaires et élargir notre compréhension du sujet.
     - **Tendances d'actualité** sur des sujets liés à l'article pour suivre l'évolution des discussions sur ces thèmes.
   
   - **Librairies utiles** :
     - `google-news-python` : Une bibliothèque qui permet de récupérer les dernières nouvelles en temps réel à partir de Google News en fonction des mots-clés ou des catégories d'articles. Cela peut compléter les informations et fournir des perspectives supplémentaires.

### 3. **YouTube**
   - **Ce que nous pourrions collecter** :
     - **Vidéos d'actualité** et analyses liées à l'article, souvent publiées par des journalistes, des blogueurs ou des experts sur des sujets spécifiques.
     - **Commentaires** des spectateurs qui peuvent offrir des perspectives ou des informations supplémentaires sur le sujet traité dans l'article.
   
   - **Librairies utiles** :
     - `google-api-python-client` : Cette bibliothèque permet d’interagir avec l'API YouTube, ce qui permet de récupérer des vidéos, titres, descriptions et commentaires associés à un sujet particulier.

Ces sources et outils nous permettraient d'enrichir les informations collectées à partir des flux RSS en explorant différents types de contenu en ligne, d'avis d'experts, et d'interactions sociales, pour une compréhension plus complète des sujets abordés dans les articles.

### Extraction d'informations

# Question 7

**Étiquettes IOB (Inside, Outside, Beginning) :**

Les étiquettes **IOB** sont un format standard utilisé dans la reconnaissance d'entités nommées (NER, Named Entity Recognition). Ce système permet de structurer les entités en fonction de leur position dans la phrase.

- **I** : **Inside** (à l’intérieur) – Indique qu’un mot fait partie d’une entité nommée, mais qu’il n’est pas au début.
- **O** : **Outside** (à l’extérieur) – Indique qu’un mot ne fait pas partie d’une entité nommée.
- **B** : **Beginning** (début) – Indique qu’un mot est le premier de l’entité nommée.

Par exemple, pour une phrase comme « Jean Dupont est maire de Plouguemeur », l'annotation IOB pourrait être :

- Jean (B-PERSON)
- Dupont (I-PERSON)
- est (O)
- maire (O)
- de (O)
- Plouguemeur (B-LOC)

Ici, **Jean Dupont** est une entité de type **PERSON**, et **Plouguemeur** est une entité de type **LOC** (lieu).

**Technique de détection des entités nommées :**

La détection des entités nommées (NER) est une tâche de traitement du langage naturel (NLP) qui vise à identifier les entités spécifiques (personnes, lieux, organisations, etc.) dans un texte. Cette tâche est souvent réalisée à l’aide de modèles d'apprentissage automatique ou de réseaux neuronaux, comme ceux utilisés par des bibliothèques populaires telles que **spaCy**.

Voici une explication simplifiée de la technique de détection des entités nommées avec spaCy :

1. **Entraînement du modèle NER** :
   - Le modèle est d'abord formé sur un large corpus de textes annotés, où les entités nommées sont identifiées manuellement. Ces annotations incluent les types d'entités (par exemple, **PERSON**, **LOC**, **ORG**) et leur position dans la phrase.
   - Le modèle apprend à reconnaître des motifs linguistiques et des contextes spécifiques qui indiquent la présence de ces entités dans de nouveaux textes.

2. **Traitement du texte avec spaCy** :
   - Une fois le modèle entraîné, on peut l'appliquer à de nouveaux textes. Le modèle scanne le texte et applique des techniques de traitement du langage telles que la **tokenisation** (diviser le texte en mots), l’**étiquetage des parties du discours** (identification de catégories grammaticales) et la **reconnaissance des entités nommées**.
   - Le modèle analyse le contexte des mots dans le texte et détermine s'ils forment une entité nommée (personne, organisation, lieu, etc.). Si tel est le cas, il les étiquette avec la catégorie appropriée (comme **PERSON**, **LOC**, ou **ORG**).

3. **Exemple d’application avec spaCy** :
   - Après avoir chargé le modèle `fr_core_news_md` (un modèle pour le français), nous pouvons l'utiliser pour analyser une phrase.
   - spaCy attribue des étiquettes IOB aux entités détectées et fournit des informations telles que la position des entités dans le texte (indices de début et de fin).

Cela permet de structurer l’information et de rendre l'extraction d'entités plus précise et plus systématique pour des applications telles que la veille économique, la création de bases de connaissances ou encore le fact-checking.

In [15]:
new_articles

{'https://www.francetvinfo.fr/france/hauts-de-france/pas-de-calais/pas-de-calais-au-moins-trois-migrants-morts-dans-une-tentative-de-traversee-de-la-manche_6983336.html#xtor=RSS-3-[general]': {'title': 'Pas-de-Calais : au moins trois migrants morts dans une tentative de traversée de la Manche',
  'date': datetime.datetime(2024, 12, 29, 14, 5, 24, tzinfo=tzoffset(None, 3600)),
  'author': 'Inconnu',
  'category': 'france',
  'content': 'Au moins trois migrants sont décédés lors d’une tentative de traversée de la Manche, dimanche 29 décembre au matin, dans le Pas-de-Calais. Ce drame porte à au moins 76 le nombre de migrants décédés cette année dans de telles tentatives.\n\nL’hélicoptère de la marine nationale a repéré une victime : un homme mort noyé en tentant de rejoindre la plage. Vers 6h15 du matin, dimanche 29 décembre, l’alerte a été donnée : une embarcation pneumatique avec une soixantaine de personnes à bord venait de chavirer. Malgré le dispositif de secours, trois personnes son

In [16]:
import spacy
# Load pipeline and related models
process = spacy.load("fr_core_news_md")
# process text with pipeline

In [17]:
commentaires = [existing_data[key]['content'] for key in existing_data.keys()]
commentaires[0]


'Beaucoup de maires et de secrétaires de mairie ont rencontré au cours de leurs mandats des problèmes de violence, essentiellement verbale. C\'est le cas de Claire. Un jour, un homme est entré dans son bureau et a frappé très fort sur son bureau pour l\'intimider. "J\'ai réagi à l\'instinct, j\'ai essayé de discuter, mais ça s\'est envenimé. Heureusement qu\'une dame est arrivée à la permanence et a menacé d\'appeler la gendarmerie", raconte Claire Carrère-Godebout, maire (SE) de Graveron-Sémerville (Eure).\n\nDes professeurs de karaté et un psychologue\n\nLa plupart de ces élus ne savent pas comment se comporter face à une situation conflictuelle. Alors, l\'Association des maires ruraux de l\'Eure les initie à l\'autodéfense, avec l\'aide de professeurs de karaté. Et un psychologue leur a expliqué comment désamorcer un différent. "C\'est nécessaire qu\'ils aient confiance en eux donc, à partir de là, je pense que cet atelier ne peut avoir que des bénéfices", explique Christophe Micaux

# QUESTION 8


In [18]:
import spacy
import json
from collections import defaultdict
import pandas as pd

# Charger le modèle spaCy pour le français (désactiver les parties inutiles pour gagner du temps)
nlp = spacy.load("fr_core_news_md", disable=["tagger", "parser"])

# Charger les données depuis le fichier JSON
with open("francetvinfo.json", "r", encoding="utf-8") as f:
    existing_data = json.load(f)

# Dictionnaire pour stocker les entités extraites par article
entities_by_article = defaultdict(list)

# Fonction pour extraire les entités
def extract_entities(doc):
    return [
        {
            "text": ent.text,
            "type": ent.label_,
            "start": ent.start_char,
            "end": ent.end_char
        }
        for ent in doc.ents
    ]

# Traitement des articles avec spaCy
for article_url, article_data in existing_data.items():
    # Traitement du contenu de chaque article avec pipe
    doc = nlp(article_data["content"])
    entities = extract_entities(doc)
    entities_by_article[article_url] = entities

# Afficher un échantillon des entités extraites pour vérifier
for article_url, entities in list(entities_by_article.items())[:2]:
    print(f"URL: {article_url}")
    for ent in entities:
        print(f" - {ent['text']} ({ent['type']})")


URL: https://www.francetvinfo.fr/sante/prevention/eure-quand-les-elus-apprennent-a-se-defendre_3791635.html#xtor=RSS-3-[france]
 - Claire (PER)
 - Claire Carrère-Godebout (PER)
 - SE (LOC)
 - Graveron-Sémerville (LOC)
 - Eure (LOC)
 - Association des maires ruraux de l'Eure (ORG)
 - Christophe Micaux (PER)
URL: https://www.francetvinfo.fr/france/video-sur-le-dos-d-un-aigle-pecheur-au-dessus-du-massif-du-mont-blanc-dont-les-glaciers-fondent-aujourd-hui-a-vue-d-oeil_3791583.html#xtor=RSS-3-[france]
 - AlertePollution Rivières (LOC)
 - Cliquez (LOC)
 - !

Embarquement (MISC)
 - Victor (MISC)
 - Pygargue (LOC)
 - Haliaeetus (LOC)
 - massif du Mont-Blanc (LOC)
 - Europe (LOC)
 - Alpes (LOC)
 - Victor (PER)
 - Jacques-Olivier Travers (PER)
 - Antarctique (LOC)
 - Amazonie (LOC)
 - Alpes (LOC)
 - glacier des Bossons (LOC)
 - Alpes (LOC)


# QUESTION 9

In [19]:
import spacy
from collections import Counter
import pandas as pd

# Chargement du modèle spaCy pour le français
process = spacy.load("fr_core_news_md", disable=["tagger"])

# Liste des commentaires à analyser
commentaires = {
    key: existing_data[key]['content']
    for key in existing_data.keys()
}

# Dictionnaire pour stocker les entités extraites par lien
resultats = {}

# Compteur global pour stocker les fréquences des entités
entite_counter = Counter()

# Compteurs distincts pour chaque type d'entité
personne_counter = Counter()
lieu_counter = Counter()
organisation_counter = Counter()

# Extraction des entités nommées
def extract_entities(doc):
    entities = []
    for ent in doc.ents:
        entities.append({
            "text": ent.text,
            "type": ent.label_,
            "start": ent.start_char,
            "end": ent.end_char
        })
        # Compter les entités par type
        if ent.label_ == "PER":  # Personnes
            personne_counter[ent.text] += 1
        elif ent.label_ == "LOC":  # Lieux
            lieu_counter[ent.text] += 1
        elif ent.label_ == "ORG":  # Organisations
            organisation_counter[ent.text] += 1
        # Compter globalement toutes les entités
        entite_counter[ent.text] += 1
    return entities

# Traitement des articles
for key, doc in zip(commentaires.keys(), process.pipe(commentaires.values(), batch_size=20, n_process=2)):
    try:
        entities = extract_entities(doc)
        resultats[key] = entities
    except Exception as e:
        print(f"Erreur lors du traitement du document {key}: {e}")

# Obtenir les 20 entités les plus fréquentes globalement
top_20_entites = entite_counter.most_common(20)

# Obtenir les 20 entités les plus fréquentes par type
top_20_personnes = personne_counter.most_common(20)
top_20_lieux = lieu_counter.most_common(20)
top_20_organisations = organisation_counter.most_common(20)

In [20]:
entities = extract_entities(doc)
resultats[key] = entities

# Exemple avec votre structure de données `resultats`
for link, entities in resultats.items():
    print(f"Article : {link}")
    
    # Itérer sur les entités extraites de l'article
    for ent in entities:
        # Affichage de l'entité et de son type
        print(f" - Texte: {ent['text']}, Type: {ent['type']}, Span: ({ent['start']}, {ent['end']})")

Article : https://www.francetvinfo.fr/sante/prevention/eure-quand-les-elus-apprennent-a-se-defendre_3791635.html#xtor=RSS-3-[france]
 - Texte: Claire, Type: PER, Span: (155, 161)
 - Texte: Claire Carrère-Godebout, Type: PER, Span: (437, 460)
 - Texte: SE, Type: LOC, Span: (469, 471)
 - Texte: Graveron-Sémerville, Type: LOC, Span: (476, 495)
 - Texte: Eure, Type: LOC, Span: (497, 501)
 - Texte: Association des maires ruraux de l'Eure, Type: ORG, Span: (653, 692)
 - Texte: Christophe Micaux, Type: PER, Span: (964, 981)
Article : https://www.francetvinfo.fr/france/video-sur-le-dos-d-un-aigle-pecheur-au-dessus-du-massif-du-mont-blanc-dont-les-glaciers-fondent-aujourd-hui-a-vue-d-oeil_3791583.html#xtor=RSS-3-[france]
 - Texte: AlertePollution Rivières, Type: LOC, Span: (1, 25)
 - Texte: Cliquez, Type: LOC, Span: (121, 128)
 - Texte: !

, Type: MISC, Span: (151, 154)
 - Texte: Victor, Type: MISC, Span: (204, 210)
 - Texte: Pygargue, Type: LOC, Span: (215, 223)
 - Texte: Haliaeetus, Type: LOC

In [21]:
# Affichage des résultats
print("Top 20 des entités globales (toutes catégories):")
top_20_df = pd.DataFrame(top_20_entites, columns=["Entité", "Fréquence"])
top_20_df.head(20)


Top 20 des entités globales (toutes catégories):


,Entité,Fréquence
0,JT,2082
1,France,811
2,Covid-19,790
3,Allemagne,569
4,Français,412
5,la France,399
6,Paris,380
7,franceinfo,365
8,Europe,332
9,Union européenne,325


In [22]:
print("\nTop 20 des entités de type PERSONNE:")
top_20_personnes_df = pd.DataFrame(top_20_personnes, columns=["Personne", "Fréquence"])
top_20_personnes_df.head(20)



Top 20 des entités de type PERSONNE:


,Personne,Fréquence
0,Emmanuel Macron,254
1,Bruno Le Maire,123
2,Angela Merkel,118
3,président de la République,63
4,Boris Johnson,55
5,Laurent Desbonnets,50
6,Muriel Pénicaud,48
7,Thierry Breton,44
8,Ursula von der Leyen,41
9,Donald Trump,40


In [23]:
print("\nTop 20 des entités de type LIEU:")
top_20_lieux_df = pd.DataFrame(top_20_lieux, columns=["Lieu", "Fréquence"])
top_20_lieux_df.head(20)


Top 20 des entités de type LIEU:


,Lieu,Fréquence
0,France,811
1,Allemagne,569
2,la France,399
3,Paris,380
4,Français,339
5,Europe,332
6,État,297
7,Royaume-Uni,260
8,Italie,256
9,Etat,237


In [24]:
print("\nTop 20 des entités de type ORGANISATION:")
top_20_organisations_df = pd.DataFrame(top_20_organisations, columns=["Organisation", "Fréquence"])
top_20_organisations_df.head(20)


Top 20 des entités de type ORGANISATION:


,Organisation,Fréquence
0,JT,2082
1,franceinfo,361
2,Union européenne,325
3,Brexit,226
4,UE,220
5,AFP,159
6,France Télévisions,128
7,Commission européenne,108
8,Renault,96
9,MAXPPP,78




### 1. **Top 20 des entités globales (toutes catégories confondues)**

Les entités globales incluent un large éventail de termes, notamment des noms géographiques, des concepts politiques et des événements sociaux. Voici quelques observations clés :

- **Les entités liées à l'actualité** : Des termes comme **"Covid-19"** (790 occurrences) et **"Brexit"** (232 occurrences) sont des entités très fréquentes, ce qui reflète l'importance de ces événements mondiaux dans les discussions actuelles. 
- **La France et l'Europe** : Des mentions récurrentes de **"France"**, **"Paris"**, **"Europe"**, et **"Union européenne"** montrent que les articles analysés se concentrent principalement sur la politique, la géopolitique et les affaires européennes. **"la France"** et **"l'Europe"** (201 occurrences) sont également mentionnées fréquemment, ce qui suggère une focalisation sur des enjeux régionaux.
- **Autres mentions géopolitiques** : **"Allemagne"** (569 occurrences), **"Chine"** (203 occurrences), **"Royaume-Uni"** (260 occurrences), et **"Italie"** (256 occurrences) témoignent des relations internationales importantes, notamment dans le contexte économique et politique européen.
- **Personnalités publiques et organisations** : Des termes comme **"Emmanuel Macron"** (254 occurrences), **"franceinfo"** (365 occurrences), et **"Covid-19"** (790 occurrences) apparaissent régulièrement, ce qui reflète leur présence notable dans les discussions médiatiques.

Ces entités indiquent que la collection de documents traite principalement des questions politiques, sociales et internationales, avec une forte concentration sur les événements liés à la pandémie de Covid-19, aux questions économiques et aux enjeux géopolitiques.

---

### 2. **Top 20 des entités de type PERSONNE**

Les entités de type **"personne"** montrent principalement des figures politiques et publiques. Voici quelques points saillants :

- **Les leaders politiques** : Les personnalités les plus fréquentes incluent des figures majeures comme **"Emmanuel Macron"** (254 occurrences), **"Angela Merkel"** (118 occurrences), **"Boris Johnson"** (55 occurrences), et **"Vladimir Poutine"** (30 occurrences). Cela reflète probablement l'importance des dirigeants politiques dans les débats et discussions sur des sujets tels que la politique nationale et internationale, le Covid-19, et la gestion des crises.
- **Les ministres et autres personnalités** : Des figures comme **"Bruno Le Maire"** (123 occurrences), **"Laurent Desbonnets"** (50 occurrences), **"Muriel Pénicaud"** (48 occurrences), et **"Thierry Breton"** (44 occurrences) soulignent l'importance des ministres et responsables d'État dans la gestion des affaires publiques et des politiques économiques.
- **Références indirectes** : Des mentions comme **"président de la République"** (63 occurrences) et **"César"** (27 occurrences) suggèrent aussi une représentation symbolique ou indirecte de figures politiques et historiques.
  
Cela montre que les documents analysés se concentrent principalement sur les figures politiques et les personnalités associées à des événements géopolitiques, économiques ou sociaux majeurs.

---

### 3. **Top 20 des entités de type LIEU**

Les entités de type **"lieu"** montrent une forte présence des pays et des régions européennes, ce qui est cohérent avec les entités globales analysées.

- **Les pays européens** : **"France"** (810 occurrences), **"Allemagne"** (569 occurrences), **"Italie"** (256 occurrences), **"Royaume-Uni"** (260 occurrences), **"Grèce"** (202 occurrences), et **"Espagne"** (132 occurrences) sont des lieux fréquemment mentionnés, ce qui indique que les documents traitent largement des questions politiques, sociales et économiques au sein de l'Europe.
- **L'Europe dans son ensemble** : Des mentions comme **"Europe"** (332 occurrences) et **"l'Europe"** (201 occurrences) témoignent de l'importance de la région dans les discussions mondiales, notamment concernant la politique et l'intégration européenne.
- **Les capitales et grandes villes** : **"Paris"** (380 occurrences), **"Bruxelles"** (158 occurrences), **"Londres"** (141 occurrences), et **"Berlin"** (135 occurrences) sont des lieux stratégiques, soulignant probablement leur importance dans les décisions politiques et économiques prises au niveau de l'UE et sur la scène mondiale.

Les lieux mentionnés sont majoritairement européens, avec une concentration particulière sur les grandes capitales et les discussions liées à l'UE, la politique et les crises géopolitiques.

---

### 4. **Top 20 des entités de type ORGANISATION**

Les entités de type **"organisation"** font ressortir des groupes médiatiques, des institutions politiques et des entreprises majeures, illustrant les structures influentes dans les domaines politique, économique et médiatique. Voici quelques observations :

- **Les médias et organisations médiatiques** : Des entités comme **"JT"** (2082 occurrences), **"franceinfo"** (361 occurrences), et **"France Télévisions"** (128 occurrences) dominent, reflétant l'importance des chaînes de télévision et des organisations médiatiques dans la diffusion des informations. La mention récurrente de **"franceinfo"** indique un rôle important dans la couverture médiatique des sujets analysés.
- **Les institutions politiques et européennes** : **"Union européenne"** (325 occurrences), **"Commission européenne"** (108 occurrences), et **"Parlement"** (41 occurrences) montrent que les discussions sont souvent centrées sur les décisions et les politiques de l'UE, ainsi que sur les discussions au niveau européen.
- **Les entreprises et organisations économiques** : Des entités comme **"Renault"** (96 occurrences), **"Airbus"** (49 occurrences), **"Alstom"** (62 occurrences), et **"Amazon"** (56 occurrences) soulignent la présence de grandes entreprises dans les discussions économiques. Cela peut être lié à des questions de politique industrielle, de croissance économique et de compétitivité internationale.
- **Les syndicats et organisations sociales** : La mention de la **"CGT"** (73 occurrences), un syndicat français majeur, et de **"Medef"** (46 occurrences), une organisation patronale, témoigne des préoccupations sociales et des tensions économiques, en particulier en France.

Les organisations extraites indiquent un mélange d'acteurs médiatiques, politiques et économiques, mettant en évidence les préoccupations sur les politiques publiques, les débats économiques et les enjeux sociaux dans la collection de documents.

---

### Conclusion générale :

Les résultats montrent que la collection de documents est fortement orientée autour des **questions géopolitiques** et **économiques** internationales, notamment celles liées à l'Europe et à la gestion de crises mondiales comme la pandémie de **Covid-19**. Les figures politiques de premier plan, notamment les présidents français et allemands, ainsi que des événements géopolitiques majeurs (comme le **Brexit** et les relations avec la **Chine**), dominent les entités extraites.

L'accent mis sur les **lieux européens**, les **organisations médiatiques**, ainsi que les **personnalités politiques** et les **organisations économiques** reflète l'intérêt pour la politique, l'économie et les relations internationales, particulièrement au sein de l'UE. Ce type d'analyse peut aider à mieux comprendre les thématiques traitées dans les documents et à identifier les principaux acteurs et lieux impliqués dans les discussions de cette période.

### Question 10

In [25]:
def count_co_occurrences(entity1, type1, entity2, type2, resultats):
    """
    Fonction qui compte le nombre de co-occurrences de deux entités données dans un même document.
    
    Parameters:
    entity1 (str) : Le texte de la première entité.
    type1 (str) : Le type de la première entité.
    entity2 (str) : Le texte de la deuxième entité.
    type2 (str) : Le type de la deuxième entité.
    resultats (dict) : Dictionnaire contenant les entités extraites pour chaque article.

    Returns:
    int : Le nombre de co-occurrences des deux entités dans un même document.
    """
    co_occurrences = 0
    
    # Parcours de tous les documents dans le résultat
    for doc_entities in resultats.values():
        # Extraire les entités du document et leur type
        entities_in_doc = [(ent['text'], ent['type']) for ent in doc_entities]
        
        # Vérifier si les deux entités (et leurs types respectifs) apparaissent dans le même document
        found_entity1 = any(ent[0] == entity1 and ent[1] == type1 for ent in entities_in_doc)
        found_entity2 = any(ent[0] == entity2 and ent[1] == type2 for ent in entities_in_doc)
        
        # Si les deux entités sont présentes dans le document, augmenter le compteur de co-occurrences
        if found_entity1 and found_entity2:
            co_occurrences += 1
    
    return co_occurrences


In [26]:
co_occurrences_count = count_co_occurrences("Emmanuel Macron", "PER", "France", "LOC", resultats)
print(f"Nombre de co-occurrences de 'Emmanuel Macron' et 'France' : {co_occurrences_count}")

Nombre de co-occurrences de 'Emmanuel Macron' et 'France' : 46


In [27]:
co_occurrences_count = count_co_occurrences("François Bayrou", "PER", "Mayotte", "LOC", resultats)

# Affichage du résultat
print(f"Nombre de co-occurrences de 'François Bayrou' et 'Mayotte' : {co_occurrences_count}")

Nombre de co-occurrences de 'François Bayrou' et 'Mayotte' : 4


### 1. Nombre de co-occurrences de **'Emmanuel Macron'** et **'France'** : **46**

**Interprétation** :
- **'Emmanuel Macron'** est une figure politique très médiatisée, étant le président actuel de la République française. Le nom de **'Emmanuel Macron'** est donc souvent mentionné dans le contexte de la **France**, que ce soit en relation avec ses décisions politiques, ses déplacements, ses discours ou ses actions à l'échelle nationale.
- Le nombre relativement élevé de **46 co-occurrences** reflète le fait qu'il est fréquemment associé au pays dans de nombreux documents et articles d'actualité. Dans un grand nombre de textes politiques, économiques, ou sociaux, **'Emmanuel Macron'** et **'France'** apparaissent ensemble, ce qui est typique des articles sur la politique française, les réformes ou les événements nationaux.
  
**Conclusion** :
- Ces 46 co-occurrences montrent que la relation entre **Emmanuel Macron** et **la France** est étroite dans les sources d'actualités analysées. Cela peut également refléter une couverture médiatique particulièrement orientée vers les actions du président à l'échelle nationale.

### 2. Nombre de co-occurrences de **'François Bayrou'** et **'Mayotte'** : **4**

**Interprétation** :
- **'François Bayrou'** est une personnalité politique française, mais sa relation avec **Mayotte** est plus spécifique. Bayrou est parfois mentionné dans le cadre de ses prises de position ou de ses actions concernant les départements d'outre-mer, et Mayotte, en tant que territoire français, peut être un sujet ponctuel dans ses discours ou dans certaines politiques publiques.
- Le nombre relativement faible de **4 co-occurrences** montre que **François Bayrou** et **Mayotte** ne sont pas fréquemment associés dans les documents analysés. Cela peut suggérer que soit les sujets impliquant **François Bayrou** ne portent pas souvent sur **Mayotte**, soit que **Mayotte** est relativement moins médiatisée dans le contexte politique de **François Bayrou**.

**Conclusion** :
- Les 4 co-occurrences peuvent signifier que l'association entre **François Bayrou** et **Mayotte** est limitée à des événements ou des décisions ponctuelles. **François Bayrou** pourrait avoir mentionné **Mayotte** dans des discours ou projets spécifiques, mais ce n'est pas un thème récurrent dans l'actualité le concernant.

### Comparaison des résultats :
- Le contraste entre les deux nombres (46 et 4) montre clairement que **'Emmanuel Macron'** et **'France'** sont beaucoup plus fréquemment associés dans les documents analysés, ce qui est logique au regard de la position centrale d'Emmanuel Macron dans la politique française et de la couverture médiatique constante qui l'entoure.
- En revanche, l'association entre **'François Bayrou'** et **'Mayotte'** est plus marginale et spécifique, ce qui est à la fois un reflet de son rôle plus restreint dans les affaires concernant ce territoire particulier, et peut-être aussi de l'importance moins médiatisée de Mayotte dans l'actualité politique liée à **François Bayrou**.

### Synthèse :
- **Emmanuel Macron** est une figure nationale avec une couverture médiatique omniprésente en lien avec la France, ce qui explique les 46 co-occurrences.
- **François Bayrou** et **Mayotte** sont davantage liés à des sujets spécifiques, et leur association dans les documents est plus marginale, d'où le faible nombre de 4 co-occurrences.



### Question 11

In [28]:
from collections import Counter

# Dictionnaire pour stocker les co-occurrences
co_occurrences = Counter()

# Fonction pour compter les co-occurrences entre paires d'entités de types différents
def count_co_occurrences(entities):
    # On utilise un dictionnaire pour stocker les entités par type
    entity_types = defaultdict(list)
    for ent in entities:
        entity_types[ent['type']].append(ent['text'])
    
    # Calcul des co-occurrences entre entités de types différents
    for type1, entities_type1 in entity_types.items():
        for type2, entities_type2 in entity_types.items():
            if type1 != type2:  # On ne compte que les entités de types différents
                for ent1 in entities_type1:
                    for ent2 in entities_type2:
                        co_occurrences[(ent1, ent2)] += 1

# Analyser toutes les entités extraites
for article_url, entities in entities_by_article.items():
    count_co_occurrences(entities)

# Afficher les 10 paires d'entités les plus fréquentes
top_co_occurrences = co_occurrences.most_common(10)
for pair, count in top_co_occurrences:
    print(f"Paire : {pair} -> Co-occurrences : {count}")


Paire : ('Auschwitz', 'Birkenau') -> Co-occurrences : 994
Paire : ('Birkenau', 'Auschwitz') -> Co-occurrences : 994
Paire : ('Covid-19', 'JT') -> Co-occurrences : 720
Paire : ('JT', 'Covid-19') -> Co-occurrences : 720
Paire : ('Royaume-Uni', 'Brexit') -> Co-occurrences : 661
Paire : ('Brexit', 'Royaume-Uni') -> Co-occurrences : 661
Paire : ('ANCV', 'ANCV') -> Co-occurrences : 608
Paire : ('ANCV', 'État') -> Co-occurrences : 598
Paire : ('État', 'ANCV') -> Co-occurrences : 598
Paire : ('Allemagne', 'JT') -> Co-occurrences : 528


### Commentaire des résultats :

1. **Paire ('Auschwitz', 'Birkenau') et ('Birkenau', 'Auschwitz')** -> **Co-occurrences : 994**

   - Ces deux entités sont souvent mentionnées ensemble, ce qui est très logique étant donné qu'**Auschwitz** et **Birkenau** désignent deux sites principaux du camp de concentration nazi pendant la Seconde Guerre mondiale, situés dans la même région et souvent associés ensemble dans les discussions historiques. Leur co-occurrence élevée peut être liée aux commémorations, aux documents historiques et aux articles éducatifs ou d'actualité relatifs à l'histoire de la Shoah.

2. **Paire ('Covid-19', 'JT') et ('JT', 'Covid-19')** -> **Co-occurrences : 720**

   - Cette paire montre que **Covid-19** et **JT (Journal Télévisé)** sont fréquemment mentionnés ensemble. Cela est tout à fait attendu, puisque la pandémie de **Covid-19** a été un sujet majeur traité en priorité dans les journaux télévisés pendant plusieurs mois, voire années. Le nombre élevé de co-occurrences témoigne de l'omniprésence de ce sujet dans les médias, en particulier dans les journaux télévisés où il est souvent au centre des discussions sur les mesures sanitaires, les mises à jour épidémiologiques et les politiques gouvernementales.

3. **Paire ('Royaume-Uni', 'Brexit') et ('Brexit', 'Royaume-Uni')** -> **Co-occurrences : 661**

   - Le **Brexit** et le **Royaume-Uni** sont étroitement liés, donc leur fréquence de co-occurrence est très élevée. Les discussions sur le **Brexit** sont inévitablement centrées sur le **Royaume-Uni**, qu'il s'agisse de négociations politiques, de ses effets économiques ou des réactions sociales. Cette paire est très logique et reflète l'importance du **Brexit** dans les actualités et analyses politiques concernant le **Royaume-Uni**.

4. **Paire ('ANCV', 'ANCV')** -> **Co-occurrences : 608**

   - Cette paire peut sembler étrange au premier abord, car elle implique la même entité mentionnée deux fois. Cependant, cela peut signifier qu'à l'intérieur des articles, l'acronyme **ANCV** (Agence Nationale pour les Chèques-Vacances) est souvent mentionné à plusieurs reprises, en particulier dans des articles économiques ou liés aux politiques de loisirs et de vacances en France. Le fait qu'elle apparaisse si souvent dans un même document est une particularité liée au contexte.

5. **Paire ('ANCV', 'État') et ('État', 'ANCV')** -> **Co-occurrences : 598**

   - **ANCV** et **État** apparaissent fréquemment ensemble, ce qui est logique si l'on considère que l'**État** (dans le sens gouvernemental) soutient souvent des initiatives comme celles de l'**ANCV**, notamment en matière de politique de vacances, d'aides sociales ou de loisirs pour les citoyens. Les co-occurrences élevées montrent que les deux entités sont fréquemment associées dans des contextes de politique publique ou de programmes gouvernementaux.

6. **Paire ('Allemagne', 'JT')** -> **Co-occurrences : 528**

   - Cette paire montre une relation entre **l'Allemagne** et le **Journal Télévisé**. Bien que l'Allemagne ne soit pas aussi centrale que certains autres pays dans le contexte des JT français, elle apparaît tout de même fréquemment dans les journaux télévisés, surtout dans le cadre de discussions sur la politique européenne, l'économie, les affaires internationales ou les crises comme la pandémie de Covid-19. Les relations diplomatiques, les événements européens ou les politiques économiques de l'Allemagne sont des sujets courants dans les journaux télévisés français.

### Synthèse générale :
- Les résultats montrent des associations logiques et attendues entre des entités fortement liées dans les discours médiatiques et politiques.
- Des entités comme **'Covid-19'**, **'Brexit'**, et **'Royaume-Uni'** apparaissent fréquemment dans des contextes médiatiques où ces événements ont été au cœur de l'actualité mondiale, expliquant ainsi les fortes co-occurrences.
- Les paires telles que **'Auschwitz'** et **'Birkenau'**, ou encore **'ANCV'** et **'État'**, mettent en évidence des relations très spécifiques qui peuvent être liées à des discussions ou des articles thématiques (histoire, politiques publiques, etc.).

Ces co-occurrences offrent un aperçu des thématiques majeures abordées dans les articles et témoignent de la manière dont certaines entités sont fréquemment liées dans les discours publics et médiatiques. Les résultats suggèrent également un fort accent sur des sujets spécifiques, comme la politique européenne, la pandémie, et des entités institutionnelles françaises.

### Question 12

In [29]:
import spacy
from collections import Counter, defaultdict
import json

# Charger le modèle spaCy pour l'analyse linguistique
nlp = spacy.load("fr_core_news_md")

# Charger le fichier JSON avec les articles
with open('francetvinfo.json', 'r', encoding='utf-8') as f:
    articles = json.load(f)

# Dictionnaire pour stocker les co-occurrences
co_occurrences = Counter()

# Fonction pour compter les co-occurrences entre entités dans la même phrase
def count_co_occurrences_in_sentence(entities_in_sentence):
    # On utilise un dictionnaire pour stocker les entités par type
    entity_types = defaultdict(list)
    for ent in entities_in_sentence:
        entity_types[ent.label_].append(ent.text)
    
    # Calcul des co-occurrences entre entités de types différents
    for type1, entities_type1 in entity_types.items():
        for type2, entities_type2 in entity_types.items():
            if type1 != type2:  # On ne compte que les entités de types différents
                for ent1 in entities_type1:
                    for ent2 in entities_type2:
                        co_occurrences[(ent1, ent2)] += 1

# Analyser tous les articles et leurs contenus
for url, article in articles.items():
    # Traiter le texte de l'article avec spaCy pour l'analyse linguistique
    doc = nlp(article['content'])

    # Pour chaque phrase du document
    for sentence in doc.sents:
        # Extraire les entités présentes dans la phrase
        entities_in_sentence = [ent for ent in sentence.ents]

        # Si des entités existent dans la phrase, compter les co-occurrences
        if entities_in_sentence:
            count_co_occurrences_in_sentence(entities_in_sentence)

# Afficher les 10 paires d'entités les plus fréquentes dans une même phrase
top_co_occurrences = co_occurrences.most_common(10)
for pair, count in top_co_occurrences:
    print(f"Paire : {pair} -> Co-occurrences : {count}")


Paire : ('France Télévisions', 'Franceinfo') -> Co-occurrences : 67
Paire : ('Franceinfo', 'France Télévisions') -> Co-occurrences : 67
Paire : ('Magazines', 'Franceinfo') -> Co-occurrences : 66
Paire : ('Franceinfo', 'Magazines') -> Co-occurrences : 66
Paire : ('France Télévisions', 'iOS & Android') -> Co-occurrences : 64
Paire : ('Franceinfo', 'iOS & Android') -> Co-occurrences : 64
Paire : ('iOS & Android', 'France Télévisions') -> Co-occurrences : 64
Paire : ('iOS & Android', 'Franceinfo') -> Co-occurrences : 64
Paire : ('Magazines', 'iOS & Android') -> Co-occurrences : 63
Paire : ('iOS & Android', 'Magazines') -> Co-occurrences : 63



### Analyse des Paires et de leurs Co-occurrences :

1. **Paires associées à 'France Télévisions' et 'Franceinfo' :**
   - Les paires **('France Télévisions', 'Franceinfo')** et **('Franceinfo', 'France Télévisions')** apparaissent chacune **67 fois**. Cela montre une forte association entre ces deux entités, suggérant qu'elles sont souvent mentionnées ensemble dans les articles. Cela pourrait être dû au fait que "France Télévisions" est l'entité mère de "Franceinfo", qui est une filiale d'information. En effet, ces deux entités pourraient être fréquemment citées ensemble dans le contexte des actualités ou des productions médias liées à France Télévisions.

2. **Paires associées à 'Franceinfo' et 'Magazines' :**
   - Les paires **('Magazines', 'Franceinfo')** et **('Franceinfo', 'Magazines')** apparaissent **66 fois**. Cette forte co-occurrence suggère que "Franceinfo" et les "Magazines" sont souvent mentionnés ensemble, probablement dans des articles où "Franceinfo" est mentionnée en rapport avec des magazines d'actualités ou des publications en ligne. Cela pourrait également signifier que des articles de presse sur Franceinfo couvrent des sujets liés à des magazines ou à des plateformes de publication.

3. **Paires associées à 'France Télévisions' et 'iOS & Android' :**
   - Les paires **('France Télévisions', 'iOS & Android')** et **('iOS & Android', 'France Télévisions')** apparaissent **64 fois**. Cela montre qu'il y a une relation notable entre France Télévisions et les plateformes mobiles (iOS et Android). Ces co-occurrences suggèrent probablement que des articles parlent des applications mobiles de France Télévisions, des plateformes où ses contenus sont accessibles, ou des sujets en lien avec l'adaptation des services médias de France Télévisions aux appareils mobiles.

4. **Paires associées à 'iOS & Android' et 'Magazines' :**
   - Les paires **('Magazines', 'iOS & Android')** et **('iOS & Android', 'Magazines')** apparaissent **63 fois**. Cela peut indiquer que dans ces articles, des magazines en ligne ou des applications dédiées aux magazines sont souvent mentionnées en relation avec les plateformes mobiles iOS et Android. Il est possible que l'analyse concerne des magazines numériques disponibles sur ces systèmes d'exploitation.

### Observations Générales :
- **Redondance des Paires :** Certaines paires sont inversées (par exemple, **('France Télévisions', 'Franceinfo')** et **('Franceinfo', 'France Télévisions')**), mais elles indiquent la même relation dans les articles. Cela montre une répétition dans la manière dont ces entités sont associées dans les phrases.
- **Thématiques Principales :** Les articles semblent fortement centrés autour des thématiques de France Télévisions, Franceinfo, les magazines et les plateformes mobiles (iOS & Android). Ces co-occurrences peuvent être liées à des articles qui abordent les applications, les plateformes numériques, ou les services en ligne de France Télévisions, en particulier dans le cadre de la consommation de médias via des appareils mobiles.

### Conclusion :
Le résultat suggère que les entités "France Télévisions", "Franceinfo", "Magazines" et "iOS & Android" sont fréquemment mentionnées ensemble, probablement en raison de leur relation étroite dans le domaine des médias numériques et des applications mobiles. Ces associations peuvent refléter la manière dont les utilisateurs accèdent aux contenus de France Télévisions et de Franceinfo, ainsi que la présence de ces entités dans les discussions sur les nouvelles technologies et les plateformes multimédia.

### Question 13

In [30]:
# Extraction des verbes entre deux entités
def extract_verbs_between_entities(doc, ent1_text, ent1_type, ent2_text, ent2_type):
    verbs = []
    ent1 = None
    ent2 = None

    # Identifier les entités dans le document
    for ent in doc.ents:
        if ent1_text in ent.text and ent.label_ == ent1_type:  # Recherche partielle
            ent1 = ent
        elif ent2_text in ent.text and ent.label_ == ent2_type:  # Recherche partielle
            ent2 = ent

    if not ent1 or not ent2:
        return verbs

    # Vérifier si les entités sont dans la même phrase
    for sent in doc.sents:
        if ent1.start >= sent.start and ent2.end <= sent.end:
            # Trier les entités pour que ent1 soit toujours avant ent2
            start, end = sorted([ent1.start, ent2.start])

            # Extraire les tokens entre les entités
            for token in sent[start:end]:
                if token.pos_ == "VERB":
                    verbs.append(token.lemma_)

    return verbs

# Traitement des articles
for key, doc in zip(commentaires.keys(), process.pipe(commentaires.values(), batch_size=20, n_process=2)):
    try:
        entities = extract_entities(doc)
        resultats[key] = entities
    except Exception as e:
        print(f"Erreur lors du traitement du document {key}: {e}")



# Recherche des relations pour les 20 entités les plus fréquentes
relations = []
for key, content in commentaires.items():
    doc = process(content)
    for ent1_text, _ in top_20_personnes:
        for ent2_text, _ in top_20_lieux:
            verbs = extract_verbs_between_entities(doc, ent1_text, "PER", ent2_text, "LOC")
            if verbs:
                relations.append((ent1_text, ent2_text, verbs))

# Afficher les relations détectées
for relation in relations:
    print(f"Relation entre {relation[0]} (PER) et {relation[1]} (LOC) : {relation[2]}")


Relation entre Emmanuel Macron (PER) et France (LOC) : ['recevoir', 'vanter']
Relation entre Emmanuel Macron (PER) et la France (LOC) : ['recevoir', 'vanter']
Relation entre Thierry Breton (PER) et État (LOC) : ['promettre', 'charger', 'vouloir', 'dire', 'décider']
Relation entre Emmanuel Macron (PER) et la France (LOC) : ['recevoir']
Relation entre président de la République (PER) et la France (LOC) : ['recevoir']
Relation entre Emmanuel Macron (PER) et Paris (LOC) : ['inaugurer']
Relation entre Edouard Philippe (PER) et Paris (LOC) : ['dédier', 'modifier']
Relation entre Jérôme Salomon (PER) et France (LOC) : ['donner']
Relation entre Emmanuel Macron (PER) et France (LOC) : ['pouvoir', 'observer', 'aller', 'passer']
Relation entre Boris Johnson (PER) et Français (LOC) : ['prévoir', 'dévoiler', 'valoir', 'avoir', 'avoir', 'écrire', 'prendre', 'reste', 'quitte', 'combler', 'choisir', 'pourrir']
Relation entre Michel Barnier (PER) et Français (LOC) : ['prévoir', 'dévoiler', 'valoir', 'a

### Question 14 : Couples d’entités et verbes associés

Voici quelques exemples de couples entités-verbes pertinents qui semblent avoir du sens :

1. **Emmanuel Macron** et **France** : 
   - Verbes associés : *recevoir*, *observer*, *commémorer*, *faire*.
   - Ces verbes reflètent le rôle présidentiel d'Emmanuel Macron dans des contextes nationaux.

2. **Boris Johnson** et **Royaume-Uni** : 
   - Verbes associés : *autoriser*, *accéder*.
   - Ces verbes illustrent des décisions ou actions politiques.

3. **Angela Merkel** et **Allemagne** : 
   - Verbes associés : *justifier*, *briguer*.
   - Ces verbes traduisent son rôle dans la politique allemande.

4. **Emmanuel Macron** et **Paris** :
   - Verbes associés : *inaugurer*, *déclarer*.
   - Cela indique des activités officielles dans la capitale.

5. **George Floyd** et **France** :
   - Verbes associés : *dénoncer*, *scander*.
   - Ces verbes montrent une mobilisation sociale en réaction à son décès.

---

### Question 15 : Utilité des couples et verbes identifiés pour trouver ou grouper des relations

1. **Détection de relations spécifiques :**
   - Les couples entité-verbe peuvent aider à identifier des interactions ou des événements spécifiques entre des entités, comme des discours, des décisions ou des mobilisations.

2. **Catégorisation des relations :**
   - Les verbes permettent de grouper les relations selon leur nature, comme :
     - **Politique** : Verbes comme *inaugurer*, *déclarer*.
     - **Sociale** : Verbes comme *dénoncer*, *scander*.
     - **Économique** : Verbes comme *autoriser*, *accéder*.

3. **Analyse temporelle ou géographique :**
   - On peut suivre l'évolution des relations dans le temps ou les localiser géographiquement.

4. **Découverte de patterns récurrents :**
   - Les verbes entre entités permettent de détecter des relations répétées, signalant des interactions ou structures importantes (ex. *président -> France -> recevoir*).

---

### Question 16 : Affiner l’analyse avec l’arbre de dépendance syntaxique

L’arbre de dépendance syntaxique pourrait être utilisé pour :
1. **Identifier des relations précises entre entités :**
   - En analysant les dépendances directes entre deux entités (sujets, objets ou compléments).
   - Exemple : Si *Emmanuel Macron* est le sujet et *observer* est le verbe gouvernant, alors *France* peut être l'objet.

2. **Réduire le bruit dans les relations :**
   - Actuellement, tous les verbes entre deux entités sont extraits. Avec l'arbre de dépendance, seuls les verbes gouvernant les entités directement pourraient être retenus.

3. **Étudier des relations indirectes ou complexes :**
   - Par exemple, dans une phrase comme :
     > *Emmanuel Macron a promis à Paris une réforme majeure*,
     - L’arbre permettrait d’extraire correctement la relation entre *Emmanuel Macron* (sujet) et *Paris* (complément).

4. **Classifier les types de relations :**
   - En combinant les relations syntaxiques et les catégories de verbes, on peut mieux comprendre la nature des interactions (ex. décisions politiques, relations interpersonnelles, etc.).

---
